In [1]:
import pandas as pd
import os
import glob
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from bioinfokit.analys import stat

sns.set_theme()
sns.set(font_scale=1)

EPSILON = 0.001
PHYS_PROPERTY = {'equate_1':'Average Diameter', 'equate_2': 'Total Surface Area', 'equate_3': 'Convex Hull'}
PHYS_PROPERTY_TO_NUM = {'Average Diameter':1, 'Total Surface Area':2, 'Convex Hull': 3}
EXPERIMENTS = ['size', 'count', 'size-count', 'count-size', 'colors', 'colors-count']
CONGRUENT_COLUMNS = ['Ratio 50 Congruent Validation Accuracy',
                     'Ratio 56 Congruent Validation Accuracy',
                     'Ratio 63 Congruent Validation Accuracy',
                     'Ratio 71 Congruent Validation Accuracy',
                     'Ratio 75 Congruent Validation Accuracy',
                     'Ratio 86 Congruent Validation Accuracy']
INCONGRUENT_COLUMNS = ['Ratio 50 Incongruent Validation Accuracy',
                     'Ratio 56 Incongruent Validation Accuracy',
                     'Ratio 63 Incongruent Validation Accuracy',
                     'Ratio 71 Incongruent Validation Accuracy',
                     'Ratio 75 Incongruent Validation Accuracy',
                     'Ratio 86 Incongruent Validation Accuracy']
STD_FILTER = 2
CONGRUENT_VALUE = 1
INCONGRUENT_VALUE = 0
CONGRUENCY = {CONGRUENT_VALUE: "Congruent", INCONGRUENT_VALUE: "Incongruent"}

In [2]:
def plot_multiple_traces(fig, data, x, y, y2, y3, y4, row, col, subtitle_index, subtitle):
    fig.add_trace(
        go.Scatter(x=data[x], y=data[y], name=y),
        row=row, col=col,
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=data[x], y=data[y2], name=y2),
        row=row, col=col,
        secondary_y=True,
    )

    fig.add_trace(
        go.Scatter(x=data[x], y=data[y3], name=y3),
        row=row, col=col,
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=data[x], y=data[y4], name=y4),
        row=row, col=col,
        secondary_y=True,
    )

    fig.layout.annotations[subtitle_index].update(text=subtitle)

In [3]:
def plot_graph(data, title, x, y, y2, color=None, is_line_plot=True):
    if color:
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        if is_line_plot:
            fig.add_trace(
            go.Scatter(x=data[x], y=data[y], name=y),
            secondary_y=False,
        )
            if y2:
                fig.add_trace(
                    go.Scatter(x=data[x], y=data[y2], name=y2),
                    secondary_y=True,
                )
        else:
            fig = px.histogram(data, x=x, y=y, color=color,  barmode='group', text_auto=True, color_discrete_sequence = px.colors.qualitative.Safe)
    else:
        if is_line_plot:
            fig = px.line(data, x=x, y=y)
        else:
            fig = px.histogram(data, x=x, y=y, color=color,  barmode='group', text_auto=True, color_discrete_sequence = px.colors.qualitative.Safe)
    fig.update_layout(
        title=title,
        xaxis_title=x,
        yaxis_title=y,)
        # coloraxis={'colorscale': 'Viridis'})
    fig.show()

In [4]:
def split_train_test_results(files_names):
    tests = []
    for file in files_names:
        if "Tested_on" in file:
            tests.append(file)
        else:
            train = file
    return tests, train

Train and test result files as a dict

In [5]:
PATH = "/Users/gali.k/phd/phd_2021/results"
result_dict = {}
for equate in PHYS_PROPERTY.keys():
    equate_dict = {}
    for experiment in EXPERIMENTS:
        curr_results_path = PATH + os.sep + equate + os.sep + experiment
        result_file_names = glob.glob(curr_results_path + os.sep + f"Results_*.csv")
        if len(result_file_names) > 0:
            tests, training_result_file_name = split_train_test_results(result_file_names)
            testing_file_name1 = tests[0]
            testing_file_name2 = tests[1]
            training_result_df =  pd.read_csv(training_result_file_name)
            testing_file_1_df =  pd.read_csv(testing_file_name1)
            testing_file_1_df['Tested_On_Equate'] = testing_file_name1[testing_file_name1.find('Tested_on'): testing_file_name1.find('AvgAccuracy')-1].replace('Tested_on_Equate', 'equate')
            testing_file_2_df =  pd.read_csv(testing_file_name2)
            testing_file_2_df['Tested_On_Equate'] = testing_file_name2[testing_file_name2.find('Tested_on'): testing_file_name2.find('AvgAccuracy')-1].replace('Tested_on_Equate', 'equate')

            equate_dict.update({experiment: { "train" : training_result_df,
                                "test_1": testing_file_1_df,
                                "test_2": testing_file_2_df}})
            result_dict.update({equate: equate_dict})

In [7]:
def add_congruency(df):
    column_names = list(df.columns)
    new_df = pd.DataFrame()
    for col in column_names:
        curr_df = pd.DataFrame()
        if "Incongruent" in col:
            curr_df[col.replace('Incongruent', '')] = df[col]
            curr_df["Congruency"] = 0
        elif "Congruent" in col:
            curr_df[col.replace('Congruent', '')] = df[col]
            curr_df["Congruency"] = 1
        else:
           curr_df[col] = df[col]
        new_df = pd.concat([new_df, curr_df])
    return new_df

Analysis #1: Trained on stimuli X and tested on Y

In [8]:
def process_df(agg_df, curr_df, trained_on, tested_on, task):
    last_gen = curr_df['Generations'].max()

    columns_to_keep = CONGRUENT_COLUMNS.copy()
    columns_to_keep.append('Subject_UID')
    cong_df = curr_df.query(f"Generations == {last_gen}")[columns_to_keep]
    cong_df['Congruency'] = CONGRUENT_VALUE

    columns_to_keep = INCONGRUENT_COLUMNS.copy()
    columns_to_keep.append('Subject_UID')
    incong_df = curr_df.query(f"Generations == {last_gen}")[columns_to_keep]
    incong_df['Congruency'] = INCONGRUENT_VALUE

    validation_acc_cong = pd.DataFrame()

    validation_acc_cong['Validation Accuracy'] = [cong_df[CONGRUENT_COLUMNS].mean().reset_index()[0].mean()]
    validation_acc_cong['Congruency'] = [CONGRUENT_VALUE]
    validation_acc_cong['Train'] = [trained_on]
    validation_acc_cong['Test'] = [tested_on]
    validation_acc_cong['Train & Test'] = ['train: ' + trained_on + '<br>' + 'test: ' + tested_on]
    validation_acc_cong['Task'] = [task]
    validation_acc_incong = pd.DataFrame()
    validation_acc_incong['Validation Accuracy'] = [incong_df[INCONGRUENT_COLUMNS].mean().reset_index()[0].mean()]
    validation_acc_incong['Congruency'] = [INCONGRUENT_VALUE]
    validation_acc_incong['Train'] = [trained_on]
    validation_acc_incong['Test'] = [tested_on]
    validation_acc_incong['Train & Test'] = ['train: ' + trained_on + '<br>' + 'test: ' + tested_on]
    validation_acc_incong['Task'] = [task]
    agg_df = pd.concat([agg_df, validation_acc_cong])
    agg_df = pd.concat([agg_df, validation_acc_incong])
    return agg_df

In [9]:
def calc_z_score_and_replace_with_mean(df, columns_list):
    for col in columns_list:
        # filter nans in order to calc zscore
        df_without_nans = df[~df[col].isna()]

        # replace nan datapoints with mean:
        abs_z_scores = np.abs(stats.zscore(df_without_nans[col]))
        mean_val = df_without_nans[col][(abs_z_scores <= STD_FILTER)].mean()
        df[col] = df[col].fillna(mean_val)

        #calc z score again:
        abs_z_scores = np.abs(stats.zscore(df[col]))
        #replace extreme values with p90
        p90_val = df[col][(abs_z_scores <= STD_FILTER)].quantile(.90)

        if (df[col][(abs_z_scores > STD_FILTER)] is not None) and (not df[col][(abs_z_scores > STD_FILTER)].empty):
            extreme_datapoints = df[col][(abs_z_scores > STD_FILTER)].values[0]
            #print(f'column {col}: replaced {extreme_datapoints} with {mean_val}')
            df[col].replace(extreme_datapoints, p90_val, inplace = True)

In [10]:
def normalize_extreme_values(df):
    # replace nans with mean
    normal_df = pd.DataFrame()
    gens = list(df['Generations'].unique())
    for gen in gens:
        cur_gen_df = df.query(f"Generations == {gen}").copy()
        acc_columns = list(cur_gen_df.filter(regex='.*Accuracy').columns)
        loss_columns = list(cur_gen_df.filter(regex='.*Loss').columns)
        calc_z_score_and_replace_with_mean(cur_gen_df, acc_columns)
        calc_z_score_and_replace_with_mean(cur_gen_df, loss_columns)
        normal_df = pd.concat([normal_df, cur_gen_df])
    return normal_df

In [ ]:
from scipy import stats
final_df = pd.DataFrame()
for phys_prop in result_dict.keys():
    print(f"Working on {phys_prop}")
    exp_result = result_dict[phys_prop]
    for exp in exp_result.keys():
        train_df = exp_result[exp]['train']
        train_df_new = normalize_extreme_values(train_df)
        final_df = process_df(final_df, train_df_new, PHYS_PROPERTY[phys_prop], PHYS_PROPERTY[phys_prop], exp)

        test1_df = exp_result[exp]['test_1']
        test1_df_new = normalize_extreme_values(test1_df)
        final_df = process_df(final_df, test1_df_new, PHYS_PROPERTY[phys_prop], PHYS_PROPERTY[test1_df['Tested_On_Equate'][0]], exp)

        test2_df = exp_result[exp]['test_2']
        test2_df_new = normalize_extreme_values(test2_df)
        final_df = process_df(final_df, test2_df_new, PHYS_PROPERTY[phys_prop], PHYS_PROPERTY[test2_df['Tested_On_Equate'][0]], exp)

Working on equate_1


/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


Working on equate_2


/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [80]:
tasks = list(final_df['Task'].unique())

Heatmaps for different physical properties

In [ ]:
congruency_vals = [0, 1]
for task in tasks:
    for cong_val in congruency_vals:
        task_df = final_df.query(f"Task == '{task}' & Congruency == {cong_val}")
        task_df = task_df[['Train', 'Test', 'Validation Accuracy']]
        df_for_heatmap = pd.pivot_table(task_df, values='Validation Accuracy', index='Train', columns='Test')
        f, ax = plt.subplots(figsize=(9, 6))
        df_for_heatmap = df_for_heatmap.round(decimals=4)
        sns.heatmap(df_for_heatmap, annot=True, linewidths=.5, ax=ax).set(title=f"Train VS Test [Task = {task}, isCongruent = {CONGRUENCY[cong_val]}]")


In [ ]:
for task in tasks:
    plot_graph(final_df.query(f"Task == '{task}'"), f'Task = {task}: Tests results when training on stimuli with different physical properties', 'Train & Test', 'Validation Accuracy',
    None, color='Congruency', is_line_plot=False)

3-way ANOVA Acc ~ C(Train) + C(Test) + C(Task)

In [84]:
congruency = 0
three_way_train_test_task_anova_df = final_df.query(f"Congruency == {congruency}")[['Validation Accuracy', 'Train', 'Test', 'Task']]
three_way_train_test_task_anova_df = three_way_train_test_task_anova_df.rename(columns={'Validation Accuracy': 'Val_Acc'})
three_way_train_test_task_anova_df = three_way_train_test_task_anova_df.reset_index().drop(columns=['index'])
model = ols("""Val_Acc ~ C(Test) + C(Train) + C(Task) +
               C(Test):C(Train) + C(Task):C(Train) + C(Task):C(Test) + C(Task):C(Test):C(Train)""", data=three_way_train_test_task_anova_df).fit()

sm.stats.anova_lm(model, typ=2)

/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning:

divide by zero encountered in double_scalars



ValueError: array must not contain infs or NaNs

In [87]:
import pingouin as pg

model1 = pg.anova(dv='Val_Acc', between=['Train','Test', 'Task'], data=three_way_train_test_task_anova_df, detailed=True)
round(model1, 2)

/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1687: RuntimeWarning:

divide by zero encountered in double_scalars



ValueError: array must not contain infs or NaNs


4-way ANOVA Acc ~ C(Congruency) + C(Train) + C(Test) + C(Task)

In [43]:
four_way_anova_df = final_df.query(f"Task == '{task}'")[['Validation Accuracy', 'Congruency', 'Train', 'Test', 'Task']]
four_way_anova_df = four_way_anova_df.rename(columns={'Validation Accuracy': 'Val_Acc'})
four_way_anova_df = four_way_anova_df.reset_index().drop(columns=['index'])
four_way_anova_df.head()

,Val_Acc,Congruency,Train,Test,Task
0,0.973333,1,Average Diameter,Average Diameter,size
1,0.907418,0,Average Diameter,Average Diameter,size
2,0.944583,1,Average Diameter,Convex Hull,size
3,0.929401,0,Average Diameter,Convex Hull,size
4,0.905583,1,Average Diameter,Total Surface Area,size


In [45]:
model = ols("""Val_Acc ~ C(Congruency) + C(Test) + C(Train) + C(Task) +
               C(Congruency):C(Test) + C(Congruency):C(Train) + C(Test):C(Train) + C(Task):C(Congruency) + C(Task):C(Train) + C(Task):C(Test) +
               C(Congruency):C(Test):C(Train) + C(Task):C(Test):C(Train) + C(Congruency):C(Task):C(Train) + C(Congruency):C(Test):C(Task) + C(Congruency):C(Test):C(Train):C(Task)""", data=four_way_anova_df).fit()

sm.stats.anova_lm(model, typ=1)

/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/statsmodels/stats/anova.py:138: RuntimeWarning:

divide by zero encountered in double_scalars



,df,sum_sq,mean_sq,F,PR(>F)
C(Congruency),1.0,3.009425e-02,0.030094,0.0,NaN
C(Test),2.0,1.415468e-02,0.007077,0.0,NaN
C(Train),2.0,1.763850e-02,0.008819,0.0,NaN
C(Task),0.0,0.000000e+00,NaN,NaN,NaN
C(Congruency):C(Test),2.0,1.411072e-02,0.007055,0.0,NaN
C(Congruency):C(Train),2.0,1.301595e-02,0.006508,0.0,NaN
C(Test):C(Train),4.0,7.213818e-03,0.001803,0.0,NaN
C(Task):C(Congruency),0.0,0.000000e+00,NaN,NaN,NaN
C(Task):C(Train),0.0,0.000000e+00,NaN,NaN,NaN
C(Task):C(Test),0.0,0.000000e+00,NaN,NaN,NaN


For each task we will do a 3-way ANOVA:
3-way ANOVA: Acc ~ C(Congruency) + C(Train) + C(Test)
2(Congruency) X 3(Train) X 3(Test)

In [46]:
task = 'size'
# for task in tasks:
three_way_anova_df = final_df.query(f"Task == '{task}'")[['Validation Accuracy', 'Congruency', 'Train', 'Test']]
three_way_anova_df = three_way_anova_df.rename(columns={'Validation Accuracy': 'Val_Acc'})
three_way_anova_df = three_way_anova_df.reset_index().drop(columns=['index'])
three_way_anova_df.head()

,Val_Acc,Congruency,Train,Test
0,0.973333,1,Average Diameter,Average Diameter
1,0.907418,0,Average Diameter,Average Diameter
2,0.944583,1,Average Diameter,Convex Hull
3,0.929401,0,Average Diameter,Convex Hull
4,0.905583,1,Average Diameter,Total Surface Area


In [86]:
#perform three-way ANOVA
model = ols("""Val_Acc ~ C(Congruency) + C(Test) + C(Train) +
               C(Congruency):C(Test) + C(Congruency):C(Train) + C(Test):C(Train) +
               C(Congruency):C(Test):C(Train)""", data=three_way_anova_df).fit()

sm.stats.anova_lm(model, typ=1)

/Users/gali.k/phd/phd_2021/venv/lib/python3.8/site-packages/statsmodels/stats/anova.py:138: RuntimeWarning:

divide by zero encountered in double_scalars



,df,sum_sq,mean_sq,F,PR(>F)
C(Congruency),1.0,3.009425e-02,0.030094,0.0,NaN
C(Test),2.0,1.415468e-02,0.007077,0.0,NaN
C(Train),2.0,1.763850e-02,0.008819,0.0,NaN
C(Congruency):C(Test),2.0,1.411072e-02,0.007055,0.0,NaN
C(Congruency):C(Train),2.0,1.301595e-02,0.006508,0.0,NaN
C(Test):C(Train),4.0,7.213818e-03,0.001803,0.0,NaN
C(Congruency):C(Test):C(Train),4.0,8.084270e-03,0.002021,0.0,NaN
Residual,0.0,1.736727e-29,inf,NaN,NaN


We did not receive values with type 2:
https://www.graphpad.com/support/faq/error-message-with-repeated-measures-two-way-anova-there-was-a-divide-by-zero-error/

two-way anova

In [52]:
two_way_test_train_anova_df = three_way_anova_df[['Train', 'Test', 'Val_Acc']]

In [53]:
model = ols("""Val_Acc ~ C(Test) + C(Train) + C(Test):C(Train)""", data=two_way_test_train_anova_df).fit()

sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Test),0.014155,2.0,0.975360,0.413618
C(Train),0.017639,2.0,1.215420,0.340988
C(Test):C(Train),0.007214,4.0,0.248542,0.903423
Residual,0.065305,9.0,NaN,NaN


Two way ANOVA between Train and Congruency --> we have effect for Congruency

In [51]:
two_way_train_cong_anova_df = three_way_anova_df[['Train', 'Congruency', 'Val_Acc']]
model = ols("""Val_Acc ~ C(Train) + C(Congruency) + C(Train):C(Congruency)""", data=two_way_train_cong_anova_df).fit()

sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
C(Train),0.017639,2.0,2.429352,0.130060
C(Congruency),0.030094,1.0,8.289766,0.013851
C(Train):C(Congruency),0.013016,2.0,1.792687,0.208345
Residual,0.043563,12.0,NaN,NaN


post hoc ? no main effect to interaction

Two-way ANOVA - Test and Congruency --> we have effect for Congruency

In [ ]:
two_way_test_congruency_anova_df = three_way_anova_df[['Test', 'Congruency', 'Val_Acc']]
model = ols("""Val_Acc ~ C(Test) + C(Congruency) + C(Test):C(Congruency)""", data=two_way_test_congruency_anova_df).fit()

sm.stats.anova_lm(model, typ=2)

post hoc ? no main effect to interaction

Two-way congruency and task

In [ ]:
TASK_NUM = {'size': 1, 'count':2, 'size-count':3, 'count-size':4, 'colors':5, 'colors-count':6}
two_way_task_congruency_anova_df = four_way_anova_df[['Task', 'Congruency', 'Val_Acc']]
two_way_task_congruency_anova_df['Task_num'] = two_way_task_congruency_anova_df['Task'].apply(lambda x: TASK_NUM[x])
model = ols("""Val_Acc ~ C(Task_num) + C(Congruency) + C(Task_num):C(Congruency)""", data=two_way_task_congruency_anova_df).fit()

sm.stats.anova_lm(model, typ=1)

* check why I have df of 0 instead of 6 in task